In [1]:
import spacy
import pyconll
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [2]:
full_train = pyconll.load_from_file('datasets/ru_gsd-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_gsd-ud-dev.conllu')

In [3]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [4]:
all_train_texts = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form if token.form is not None  else '' for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[10:20]))
print('\n'.join(all_train_labels[10:20]))

В ходе этого обширного наступления была освобождена территория Белоруссии , восточной Польши и часть Прибалтики и практически полностью разгромлена германская группа армий `` Центр &#39;&#39; .
Член Русского ПЕН - центра .
Название дано европейцами ближайшему к ним восточному региону .
С 1986 года эсминец `` Боевой &#39;&#39; был включён в состав 175-ой бригады ракетных кораблей 10-ой опэск ( оперативной эскадры ) Тихоокеанского флота ВМФ СССР .
22 июня 1987 года корабль начал переход на Тихоокеанский флот с несением боевой службы в Персидском заливе ( провёл 22 судна в 16 конвоях ) .
Отличился во время освобождения Белорусской ССР .
С начала Великой Отечественной войны -- на её фронтах .
Расположена в центральной части области в 5 км к юго-востоку от Ярцева , в 5 км южнее автодороги , на берегу реки Великая .
Фрэ́нсис Бэй ( ) , в девичестве -- Го́ффман ( ; 23 января 1919 `` Альберта , Канада -- 15 сентября 2011 `` Лос-Анджелес , Калифорния , США ) -- канадо-американская актриса .
Варт

In [5]:
test_sent = all_test_texts[15]
test_data = fdata_test

In [6]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(test_sent.lower().split()), unigram_tagger.evaluate(test_data))

[('в', 'ADP'),
 ('передаче', 'NOUN'),
 ('приняли', 'VERB'),
 ('участие', 'NOUN'),
 ('поэт', 'NOUN'),
 ('андрей', None),
 ('вознесенский', None),
 (',', 'PUNCT'),
 ('художник', 'NOUN'),
 (',', 'PUNCT'),
 ('модельер', None),
 ('вячеслав', None),
 ('зайцев', None),
 (',', 'PUNCT'),
 ('композитор', 'NOUN'),
 ('родион', None),
 ('щедрин', None),
 ('.', 'PUNCT')]

0.7109061405756256

In [7]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.tag(test_sent.lower().split()), bigram_tagger.evaluate(test_data))

[('в', 'ADP'),
 ('передаче', 'NOUN'),
 ('приняли', 'VERB'),
 ('участие', 'NOUN'),
 ('поэт', 'NOUN'),
 ('андрей', None),
 ('вознесенский', None),
 (',', 'PUNCT'),
 ('художник', 'NOUN'),
 (',', 'PUNCT'),
 ('модельер', None),
 ('вячеслав', None),
 ('зайцев', None),
 (',', 'PUNCT'),
 ('композитор', 'NOUN'),
 ('родион', None),
 ('щедрин', None),
 ('.', 'PUNCT')]

0.7122726108121957

In [8]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

[('в', 'ADP'),
 ('передаче', None),
 ('приняли', None),
 ('участие', None),
 ('поэт', None),
 ('андрей', None),
 ('вознесенский', None),
 (',', None),
 ('художник', None),
 (',', None),
 ('модельер', None),
 ('вячеслав', None),
 ('зайцев', None),
 (',', None),
 ('композитор', None),
 ('родион', None),
 ('щедрин', None),
 ('.', None)]

0.06482193184729695

In [142]:
trigram_tagger = TrigramTagger(fdata_train, backoff=unigram_tagger)
display(trigram_tagger.tag(test_sent.lower().split()), trigram_tagger.evaluate(test_data))

[('в', 'ADP'),
 ('передаче', 'NN'),
 ('приняли', 'NN'),
 ('участие', 'NN'),
 ('поэт', 'NN'),
 ('андрей', 'NNP'),
 ('вознесенский', 'NN'),
 (',', 'NN'),
 ('художник', 'NN'),
 (',', 'NN'),
 ('модельер', 'NN'),
 ('вячеслав', 'NNP'),
 ('зайцев', 'NN'),
 (',', 'NN'),
 ('композитор', 'NN'),
 ('родион', 'NNP'),
 ('щедрин', 'NN'),
 ('.', 'NN')]

0.06482193184729695

In [103]:
import codecs
fileObj = codecs.open( 'names_table.jsonl', "r", "utf_8_sig" )
names = fileObj.read() # или читайте по строке
fileObj.close()

fileObj = codecs.open( 'midnames_table.jsonl', "r", "utf_8_sig" )
family = fileObj.read() # или читайте по строке
fileObj.close()

fileObj = codecs.open( 'surnames_table.jsonl', "r", "utf_8_sig" )
second_family = fileObj.read() # или читайте по строке
fileObj.close()

In [104]:
c = ''
k = names.split(':')
for i in range(len(k)):
    if k[i] == '{"text"' or   k[i] == ' 100.0}\r\n{"text"':
        c += ' ' + k[i+1]

k = c.split(', "regorg"  ')
k = ' '.join(l for l in k if (l != '"' and l != "'" and l != ","))
k = k.split('" "')
k = ' '.join(l for l in k if (l != '"' and l != "'" and l != ","))
k = k.lower()
k

'  "александр сергей владимир андрей алексей дмитрий елена татьяна евгений николай наталья ольга юрий игорь михаил ирина виктор светлана олег валерий анатолий людмила галина павел иван максим марина анна вячеслав юлия валентина денис роман екатерина константин надежда виталий василий мария любовь геннадий антон вадим нина оксана лариса илья анастасия борис руслан станислав владислав вера петр леонид артем наталия евгения эдуард тамара виктория кирилл александра лидия григорий георгий артур валентин алла никита инна раиса лилия маргарита ксения марат олеся дарья альберт антонина федор зинаида яна тимур кристина егор жанна зоя аркадий  алена ринат артём альбина алина эльвира лев магомед елизавета валерия ильдар степан вероника ярослав рамиль даниил семен а. полина алевтина диана римма шамиль герман анжелика яков роберт алёна радик данил дамир рустем наиль айрат тимофей армен глеб равиль давид арсен роза гульнара клавдия анжела ренат марк таисия рашид софья азат альфия дина карина рафаэль

In [105]:
c = ''
k = names.split(':')
for i in range(len(k)):
    if k[i] == '{"text"' or   k[i] == ' 100.0}\r\n{"text"':
        c += ' ' + k[i+1]

k = c.split(', "regorg"  ')
k = ' '.join(l for l in k if (l != '"' and l != "'" and l != ","))
k = k.split('" "')
k = ' '.join(l for l in k if (l != '"' and l != "'" and l != ","))
k = k.lower()
k = ''.join(l for l in k if (l != '"' and l != "'" and l != ","))

In [106]:
names = k

In [107]:
c = ''
k = family.split(':')
for i in range(len(k)):
    if k[i] == '{"text"' or   k[i] == ' "m"}\r\n{"text"':
        c += ' ' + k[i+1]

k = c.split(', "regorg"  ')
k = ' '.join(l for l in k if (l != '"' and l != "'" and l != ","))
k = k.split('" "')
k = ' '.join(l for l in k if (l != '"' and l != "'" and l != ","))
k = k.lower()
k = ''.join(l for l in k if (l != '"' and l != "'" and l != ","))
k

'  владимирович александрович николаевич викторович сергеевич александровна анатольевич николаевна михайлович васильевич алексеевич викторовна анатольевна васильевна геннадьевич юрьевна  павлович вячеславович леонидович игоревич петровна валентинович валерьевна григорьевич евгеньевна витальевич борисовна павловна вячеславовна георгиевна семенович яковлевич вадимович львович ильич эдуардович аркадьевич владиславович степановна семеновна станиславовна ильинична магомедович романович вадимовна эдуардовна львовна германович владиславовна афанасьевич ефимович антонович валериевна маратович русланович вениаминовна филиппович гаврилович геннадиевна кузьмич афанасьевна фаритович иосифовна матвеевич германовна маркович рафикович данилович алиевич антоновна ефимовна иннокентьевич равильевич раисович давидович рашидович магомедовна феликсович гавриловна хасанович ахмедович тихонович никитович матвеевна анварович ильдарович мансурович серафимович радикович рамазанович абрамович фёдорович кузьминич

In [108]:
familys = k

In [109]:
c = ''
k = second_family.split(':')
for i in range(len(k)):
    if k[i] == '{"text"' or   k[i] == ' "m"}\r\n{"text"':
        c += ' ' + k[i+1]

k = c.split(', "regorg"  ')
k = ' '.join(l for l in k if (l != '"' and l != "'" and l != ","))
k = k.split('" "')
k = ' '.join(l for l in k if (l != '"' and l != "'" and l != ","))
k = k.lower()
k = ''.join(l for l in k if (l != '"' and l != "'" and l != ","))
k

'  иванов хайруллин баева андросов курочкина панасенко литвин букреев олейникова земцов трубников гребенщиков данилина соловей асланян волчков самигуллин соболь лисина есина дудников преображенский садовникова кучина сухинин золин логвинова нагиев петракова косенкова ширшова калюжная полонский милютина краснощеков бабенков луканин жучкова мурашкина гетманов пшеничникова кострюков козулин грунин панюков боголюбов мосеев мичурин хорева выдрин рябенко эпштейн евдокименко шальнов чепелева старов маричев мисюра хизриев костерина яскевич северина салиев вавилина родченко рябухина брук чечулина амерханов бояршинова низов ухин кузьменкова скородумова малофеева ананян бобина сагателян лысюк крот шаболдин терещенков сахненко чернюк джалилова булаева басаргин калитин зайнутдинова лунгу цепляев гринчук сибачев хацкевич цапко докторов яцун марчуков певнев старицын пальчикова равилов манякин радыгин тодоров богунов папков фотин жмаев аленичев шокин кипкеев юриков безрук романец бучнева еремкина напа

In [111]:
 second_familys = k

In [112]:
Aliesys = second_familys + familys + names

In [90]:
k = family.split(':')
k[:3000]

['{"text"',
 ' "Владимирович", "regorg"',
 ' {"78"',
 ' 96855, "60"',
 ' 4473, "51"',
 ' 6019, "47"',
 ' 11525, "77"',
 ' 271843, "50"',
 ' 65521, "23"',
 ' 40192, "68"',
 ' 6854, "91"',
 ' 7342, "61"',
 ' 28004, "13"',
 ' 4010, "63"',
 ' 33977, "66"',
 ' 51993, "86"',
 ' 9185, "34"',
 ' 19236, "37"',
 ' 10508, "16"',
 ' 18133, "89"',
 ' 3281, "14"',
 ' 4976, "40"',
 ' 7509, "71"',
 ' 10539, "74"',
 ' 30327, "54"',
 ' 46061, "27"',
 ' 13098, "62"',
 ' 8613, "02"',
 ' 13942, "32"',
 ' 6314, "33"',
 ' 10793, "39"',
 ' 12283, "46"',
 ' 6205, "45"',
 ' 4615, "72"',
 ' 13499, "75"',
 ' 4457, "55"',
 ' 15608, "52"',
 ' 30875, "73"',
 ' 8375, "92"',
 ' 2542, "36"',
 ' 15495, "25"',
 ' 19028, "48"',
 ' 6577, "31"',
 ' 10325, "03"',
 ' 4752, "19"',
 ' 2954, "38"',
 ' 17755, "53"',
 ' 3903, "56"',
 ' 10755, "26"',
 ' 13193, "05"',
 ' 447, "22"',
 ' 18328, "24"',
 ' 23189, "44"',
 ' 4622, "15"',
 ' 1601, "43"',
 ' 10027, "59"',
 ' 21659, "41"',
 ' 3229, "01"',
 ' 1563, "07"',
 ' 1437, "76"',
 ' 1

In [131]:
from nltk.tag import SequentialBackoffTagger

class NamesTagger(SequentialBackoffTagger):
    
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([name.lower() for name in Aliesys.split()])
        
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word.lower() in self.name_set:
            return "NNP"
        else:
            return 'NN'
        
nt = NamesTagger()
print(nt.tag(test_sent.lower().split()))
# print(nt.tag(['Дом']))

[('в', 'NN'), ('передаче', 'NN'), ('приняли', 'NN'), ('участие', 'NN'), ('поэт', 'NN'), ('андрей', 'NNP'), ('вознесенский', 'NN'), (',', 'NN'), ('художник', 'NN'), (',', 'NN'), ('модельер', 'NN'), ('вячеслав', 'NNP'), ('зайцев', 'NN'), (',', 'NN'), ('композитор', 'NN'), ('родион', 'NNP'), ('щедрин', 'NN'), ('.', 'NN')]


In [167]:
from nltk.tag import TrigramTagger

def backoff_tagger(train_sents, tagger_classes, backoff = None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff = backoff)
    return backoff

backoff = NamesTagger()
tag =  backoff_tagger(fdata_train, 
                      [TrigramTagger, BigramTagger, UnigramTagger],
                      backoff = backoff)
tag.evaluate(test_data)

0.1960030745580323

In [170]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        if (tok is None or tok[0] is None):
          continue
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        if (tok is None or tok[0] is None):
          continue
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [171]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [172]:
test_enc_labels = le.transform(test_label)

In [173]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char_wb', n_features=500)
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

In [174]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

C:\Users\alimc\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [175]:
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.8546417285848492

In [177]:
hvectorizer = CountVectorizer(ngram_range=(1, 3), analyzer='char_wb')
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)
# tfivec = TfidfVectorizer()
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

C:\Users\alimc\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9276624818515672

In [178]:
hvectorizer = TfidfVectorizer(ngram_range=(1, 1), analyzer='word')
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)
# tfivec = TfidfVectorizer()
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

C:\Users\alimc\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5909129729268084

Выводы:

1) UnigramTagger - 0.878
2) BigramTagger - 0.710
3) BigramTagger + UnigramTagger - 0.884
4) TrigramTagger - 0.407
5) TrigramTagger + BigramTagger + UnigramTagger - 0.883 (но чуть лучше, чем Trigram+Unigram)
6) Свой (HashingVectorizer(char, 1-3, 500 feat) + LR 100 итераций) - 0.873
7) Свой (HashingVectorizer(char, 1-5, max feat) + LR 100 итераций) - 0.931
8) Свой (TfidfVectorizer(char, 1-3, max feat) + LR 100 итераций) - 0.93
9) Свой (TfidfVectorizer(word, 1-1, max feat) + LR 100 итераций) - 0.661

Общий вывод: наилучшие результаты даёт комбинация Unigram+Bigram и линейная регрессия с любым векторизатором без ограничения на кол-во фичей. Линейная регрессия работает при этом заметно медленнее, но даёт +5% точности. Попытки векторизировать данные не по символам, а по словам приводят к сильному ухудшению точности. Можно предположить, что это связано с тем, что при использовании символьных n-грам, модель может научиться понимать незнакомые слова по "похожести" на присутствующие в тренировочном наборе, в том числе по окончаниям и т.п., что даёт ей шанс верно угадать тэг для таких слов, в то время как модель, обучающаяся на словах, не имеет никаких данных о строение слова, и увидев новое слово не понимает, на что оно может быть похоже.